In [7]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, text

# 데이터베이스 연결 설정
def connect_to_database():
    # 데이터베이스 연결 정보
    username =   
    password = '  
    hostname = 
    port = 
    database = 
    
    # SQLAlchemy 엔진 생성
    engine = create_engine(f'mysql+pymysql://{username}:{password}@{hostname}:{port}/{database}')
    return engine

In [8]:
def fetch_data(engine):
    fetch_query = """
    SELECT tbl_user_sub.user_pid, tbl_user_sub.zip AS user_zip, tbl_user_sub.adres AS user_adres, tbl_user_sub.dtl_adres AS user_dtl_adres,
           tbl_shop_sub.str_shop_pid, tbl_shop_sub.str_shop_code, tbl_shop_sub.str_shop_nm,
           tbl_shop_sub.zip AS shop_zip, tbl_shop_sub.adres AS shop_adres, tbl_shop_sub.dtl_adres AS shop_dtl_adres
    FROM (
        SELECT thai.user_pid, thai.zip, thai.adres, thai.dtl_adres
        FROM tbl_hr_add_info AS thai
        JOIN tbl_user AS tu
        ON thai.user_pid = tu.user_pid
    ) AS tbl_user_sub
    JOIN (
        SELECT tss.str_shop_pid, tss.str_shop_code, tss.str_shop_nm, tss.zip, tss.adres, tss.dtl_adres, tss.la, tss.lo, tusr.user_pid
        FROM tbl_user_store_rel AS tusr
        JOIN tbl_store_shop AS tss
        ON tusr.str_shop_pid = tss.str_shop_pid
    ) AS tbl_shop_sub
    ON tbl_user_sub.user_pid = tbl_shop_sub.user_pid
    """
    return pd.read_sql(text(fetch_query), engine)

def insert_data_into_dataframe(data):
    df = pd.DataFrame(data)
    return df

def main():
    engine = connect_to_database()  # DB에 연결하는 함수
    data = fetch_data(engine)
    df = insert_data_into_dataframe(data)
    print(f"{len(df)} rows fetched and converted to DataFrame.")
    print(df)
    return df  # DataFrame 반환

if __name__ == "__main__":
    main()
    

219 rows fetched and converted to DataFrame.
     user_pid user_zip          user_adres user_dtl_adres  str_shop_pid  \
0           9    04776     서울 성동구 성수이로 2-1        (성수동2가)           188   
1         146                                                       188   
2         147     None                None           None           203   
3         148     None                None           None           203   
4         152    22222                  주소             상세           206   
..        ...      ...                 ...            ...           ...   
214       441    13479  경기 성남시 분당구 서판교로 32           102호           479   
215       441    13479  경기 성남시 분당구 서판교로 32           102호           480   
216       381    04782     서울 성동구 연무장5가길 7        (성수동2가)           479   
217       381    04782     서울 성동구 연무장5가길 7        (성수동2가)           480   
218       417     null                null           null           304   

    str_shop_code str_shop_nm shop_zip        shop_adr

In [9]:
import json
import pandas as pd
import urllib
import IPython
import urllib.request
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError

option='traoptimal' #최적 거리 계산
client_id = 'uo2xxmprpi'  #api 정보
client_secret = 'DGA3bnsb1NgcAUA5CJAjtwlksM8O1281OUcRNbpu' 

# API 호출 및 실제 이동 거리 계산
def get_optimal_route(start, goal, option=option):
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={start[0]},{start[1]}&goal={goal[0]},{goal[1]}&option={option}"
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    
    try:
        response = urllib.request.urlopen(request)
        res = response.getcode()
        
        if res == 200:  # API 응답 정상 코드
            response_body = response.read().decode('utf-8')
            data = json.loads(response_body)
            if 'route' in data and 'traoptimal' in data['route']:
                travel_time = data['route']['traoptimal'][0]['summary']['duration']
                return travel_time
            else:
                print("경로 데이터를 찾을 수 없습니다.")
                return None
        else:
            print(f"API 응답 오류: {res}")
            return None
            
    except HTTPError as e:
        print(f"HTTP 오류 발생: {e.code} - {e.reason}")
    except URLError as e:
        print(f"URL 오류 발생: {e.reason}")
    except Exception as e:
        print(f"예상치 못한 오류 발생: {str(e)}")
    return None

# 주소 -> 위경도 변환
def get_location(loc):
    url = f"https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=" + urllib.parse.quote(loc)
    
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    
    try:
        response = urlopen(request)
        res = response.getcode()
    
        if res == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)
            if 'addresses' in response_body and len(response_body['addresses']) > 0:
                lat = response_body['addresses'][0]['y']
                lon = response_body['addresses'][0]['x']
                return (lon, lat)
            else:
                print('주소를 찾을 수 없습니다.')
                return None, None
        
        else:
            print(f"API 응답 오류: {res}")
            return None, None
            
    except HTTPError as e:
        print(f"HTTP 오류 발생: {e.code} - {e.reason}")
    except URLError as e:
        print(f"URL 오류 발생: {e.reason}")
    except Exception as e:
        print(f"예상치 못한 오류 발생: {str(e)}")
    return None, None

# 실제 데이터프레임 연산 함수
def geocode_df(df):
    df = df.copy()
    df.loc[:, 'user_lat'] = None
    df.loc[:, 'user_lng'] = None
    df.loc[:, 'shop_lat'] = None
    df.loc[:, 'shop_lng'] = None
    df.loc[:, 'travel_time'] = None

    for index, row in df.iterrows():
        if pd.notnull(row['user_adres']):
            user_lat, user_lng = get_location(row['user_adres'])
            df.loc[index, 'user_lat'] = user_lat
            df.loc[index, 'user_lng'] = user_lng
            print(f"User address: {row['user_adres']} -> Lat: {user_lat}, Lng: {user_lng}")
        if pd.notnull(row['shop_adres']):
            shop_lat, shop_lng = get_location(row['shop_adres'])
            df.loc[index, 'shop_lat'] = shop_lat
            df.loc[index, 'shop_lng'] = shop_lng
            print(f"Shop address: {row['shop_adres']} -> Lat: {shop_lat}, Lng: {shop_lng}")
            
        if pd.notnull(df.loc[index, 'user_lat']) and pd.notnull(df.loc[index, 'shop_lat']):
            start = get_location(row['user_adres'])
            goal = get_location(row['shop_adres'])
            print(f"Calculating travel time from {start} to {goal}")
            travel_time = get_optimal_route(start, goal, option=option)
            print(f"Travel time: {travel_time}")
            df.loc[index, 'travel_time'] = travel_time
        else:
            print(f"Skipping index {index} due to missing coordinates.")
    return df


In [10]:
def main():
    engine = connect_to_database()  # DB에 연결하는 함수
    data = fetch_data(engine)
    df = insert_data_into_dataframe(data)
    df_dn = df.dropna(subset=['user_adres', 'shop_adres'])
    # 주소 지오코딩 및 소요 시간 계산
    df_geocoded = geocode_df(df_dn)
    df_geocoded_dn = df_geocoded.dropna(subset=['travel_time'])
    # 결과를 Jupyter Notebook에서 보기
    IPython.display.display(df_geocoded_dn)
    df_geocoded_dn.to_csv('data.csv', index=True, encoding='utf-8-sig')

if __name__ == "__main__":
    main()

User address: 서울 성동구 성수이로 2-1 -> Lat: 127.0537224, Lng: 37.5355643
주소를 찾을 수 없습니다.
Shop address: 서울시 광진구 성동로1길 -> Lat: None, Lng: None
Skipping index 0 due to missing coordinates.
주소를 찾을 수 없습니다.
User address:  -> Lat: None, Lng: None
주소를 찾을 수 없습니다.
Shop address: 서울시 광진구 성동로1길 -> Lat: None, Lng: None
Skipping index 1 due to missing coordinates.
주소를 찾을 수 없습니다.
User address: 주소 -> Lat: None, Lng: None
Shop address: 경기도 양주시 은현면 -> Lat: 127.024525, Lng: 37.874999
Skipping index 4 due to missing coordinates.
주소를 찾을 수 없습니다.
User address: 111111 -> Lat: None, Lng: None
Shop address: 경기도 양주시 은현면 -> Lat: 127.024525, Lng: 37.874999
Skipping index 5 due to missing coordinates.
주소를 찾을 수 없습니다.
User address:  -> Lat: None, Lng: None
Shop address: 경기도 양주시 은현면 -> Lat: 127.024525, Lng: 37.874999
Skipping index 6 due to missing coordinates.
주소를 찾을 수 없습니다.
User address:  -> Lat: None, Lng: None
Shop address: 경기도 양주시 은현면 -> Lat: 127.024525, Lng: 37.874999
Skipping index 7 due to missing coordinates.
주소를 찾을 

,user_pid,user_zip,user_adres,user_dtl_adres,str_shop_pid,str_shop_code,str_shop_nm,shop_zip,shop_adres,shop_dtl_adres,user_lat,user_lng,shop_lat,shop_lng,travel_time
14,177,04782,서울 성동구 연무장5길 9-16,(성수동2가) 506호,231,CDM00231,울산방어점,44086,울산 동구 방어동 912-4,1층,127.0526591,37.5444391,129.4191149,35.4900514,16201243
18,187,04782,서울 성동구 연무장5길 7,가나다,238,CDM00238,남양주별내점,12084,경기 남양주시 별내면 청학로중앙길 15,,127.0519899,37.5437395,127.1178956,37.7086755,2595744
20,188,11427,경기도 양주시 화합로 941번길 115,주택,239,CDM00239,나주빛가람점,58217,전남 나주시 상야4길 22,(빛가람동),127.0274189,37.8658872,126.7917513,35.0227487,15588062
23,177,04782,서울 성동구 연무장5길 9-16,(성수동2가) 506호,238,CDM00238,남양주별내점,12084,경기 남양주시 별내면 청학로중앙길 15,,127.0526591,37.5444391,127.1178956,37.7086755,2588264
25,188,11427,경기도 양주시 화합로 941번길 115,주택,237,CDM00237,부산아트몰링점,49311,부산 사하구 낙동남로 1413,"(하단동, 아트몰링)",127.0274189,37.8658872,128.9664750,35.1067550,16340006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,436,04770,서울 성동구 강변북로 257,,483,home-01,군자점,05003,서울 광진구 광나루로 343,(군자동),127.0415922,37.5389565,127.0681453,37.5485635,1326319
214,441,13479,경기 성남시 분당구 서판교로 32,102호,479,home-ch_01,강동점,05200,서울 강동구 가래여울길 3,(강일동) 5층,127.0978997,37.3897823,127.1688665,37.5762402,2918678
215,441,13479,경기 성남시 분당구 서판교로 32,102호,480,home-ch_02,군자점,05003,서울 광진구 광나루로 343,(군자동),127.0978997,37.3897823,127.0681453,37.5485635,3110356
216,381,04782,서울 성동구 연무장5가길 7,(성수동2가),479,home-ch_01,강동점,05200,서울 강동구 가래여울길 3,(강일동) 5층,127.0536274,37.5440703,127.1688665,37.5762402,2323366
